**Dependencies**

In [ ]:
library(data.table)
library(limma)
library(DMRcate)
library(minfi)
library(GenomicRanges)
library(RColorBrewer)
library(IlluminaHumanMethylationEPICanno.ilm10b4.hg19)

**Loading data and metadata**

In [3]:
#  Illumina Infinium MethylationEPIC Beadchip data from 473 positive and 101 negative SARS-CoV-2 individuals 
GSE179325<-fread("path",header=T)  # check folder's readme to find data address
GSE179325 <- as.data.frame(GSE179325)
rownames(GSE179325) <- GSE179325$ID

# Converting beta values to M-values 
beta_values <- GSE179325[, -1] 
m_values <- log2(beta_values / (1 - beta_values))

In [15]:
# metadata list
metaGSE179325_list <- readLines("path") # check folder's readme to find data address
length(metaGSE179325_list)
# Severity inf: element 42 in list
metadata_string <- metaGSE179325_list[[42]]
metadata_split <- strsplit(metadata_string, "\t")[[1]]
disease_states <- metadata_split[-1]
disease_states <- gsub("disease state: ", "", disease_states)
disease_states <- trimws(disease_states)
head(disease_states)
length(disease_states)  
table(disease_states)  

[1] 69

[1] "\"SEVERE\"" "\"MILD\""   "\"MILD\""   "\"SEVERE\"" "\"MILD\""  
[6] "\"SEVERE\""

[1] 574

disease_states
    "MILD" "NEGATIVE"   "SEVERE" 
       360        101        113 

In [16]:
sample_info<-cbind(colnames(GSE179325[, -1]),disease_states)
colnames(sample_info)<- c("Sample_Name","Group")
sample_info<- as.data.frame(sample_info)

,Sample_Name,Group
,<chr>,<chr>
1,100120,"""SEVERE"""
2,100165,"""MILD"""
3,101417,"""MILD"""
4,101573,"""SEVERE"""
5,103811,"""MILD"""
6,104111,"""SEVERE"""


**Data distribution visualisation**

In [23]:
#beta value distribution plot
pdf("beta_values_325.pdf", width = 7, height = 7)  
beta_values <- as.matrix(GSE179325[, -1])
unique_groups <- unique(sample_info$Group)
num_groups <- length(unique_groups)
colors <- brewer.pal(max(3, num_groups), "Dark2") 
par(mfrow = c(1, 1))
plot(NULL, xlim = range(beta_values, na.rm = TRUE), ylim = c(0, 6), 
     main = "Beta-Values Distribution", xlab = "Beta-Value", ylab = "Density")
for (i in seq_along(unique_groups)) { group <- unique_groups[i]
  group_beta_values <- beta_values[, sample_info$Group == group]  
  density_curve <- density(as.vector(group_beta_values), na.rm = TRUE)  
  lines(density_curve, col = colors[i], lwd = 2) }
legend("topright", legend = unique_groups, col = colors[1:num_groups], 
       lwd = 2, box.lwd = 0, cex = 0.8)
grid()
dev.off()

#M value distribution plot
pdf("m_values_325.pdf", width = 7, height = 7)
m_values[m_values == Inf | m_values == -Inf] <- NA
plot(NULL, xlim = range(m_values, na.rm = TRUE), ylim = c(0, 0.4), 
     main = "M-Values Distribution", xlab = "M-Value", ylab = "Density")
for (i in seq_along(unique_groups)) {
  group <- unique_groups[i]
  group_m_values <- m_values[, sample_info$Group == group]
  density_curve <- density(as.vector(group_m_values), na.rm = TRUE)
  lines(density_curve, col = colors[i], lwd = 2)}
legend("topright", legend = unique_groups, col = colors[1:num_groups], 
       lwd = 2, box.lwd = 0, cex = 0.8)
grid()
dev.off()

png 
  2

**Making design matrix**

In [17]:
disease_states_clean <- gsub('^"|"$', '', disease_states)
disease_states_factor <- factor(disease_states_clean, levels = c("NEGATIVE", "MILD", "SEVERE"))
design <- model.matrix(~ 0 + disease_states_factor)
colnames(design) <- levels(disease_states_factor)
print(head(design))
print(dim(design)) # Should be 574 x 3

# contrasts
contrast_matrix <- makeContrasts(
    SEVEREvsNEGATIVE = SEVERE - NEGATIVE,
    SEVEREvsMILD = SEVERE - MILD,
    levels = design)
contrast_matrix

  NEGATIVE MILD SEVERE
1        0    0      1
2        0    1      0
3        0    1      0
4        0    0      1
5        0    1      0
6        0    0      1
[1] 574   3


,SEVEREvsNEGATIVE,SEVEREvsMILD
NEGATIVE,-1,0
MILD,0,-1
SEVERE,1,1


**Finding DMPs**

In [18]:
# DMP with limma _ Fitting linear model
fit <- lmFit(m_values, design)
fit <- contrasts.fit(fit, contrast_matrix)
fit <- eBayes(fit)

# DMPs for each contrast
contrasts <- colnames(contrast_matrix)
dmp_list <- list()

for (contrast in contrasts) {
  dmp <- topTable(fit, coef = contrast, number = Inf, adjust.method = "fdr")
  dmp_list[[contrast]] <- dmp
  write.csv(dmp, file = paste0("DMPs_", contrast, ".csv"))
}


**Finding DMRs**

In [ ]:

# DMRs with DMRcate
m_values <- as.matrix(m_values)
my_annotation <- cpg.annotate(
  object = m_values,
  datatype = "array",
    what = "M", 
  analysis.type = "differential",
  design = design,
  contrasts = TRUE,
  coef = colnames(contrast_matrix),
    arraytype = "EPICv1",
    anno = anno)

# DMRs for each contrast
dmr_results <- list()
for (contrast in contrasts) {
  dmrcoutput <- dmrcate(
    my_annotation,
    lambda = 500,  # Bandwidth for smoothing
    C = 2,          # Scaling factor
    cutoff = 0.05   # FDR cutoff
  )
  dmr_results[[contrast]] <- dmrcoutput
  write.csv(dmrcoutput$results, file = paste0("DMRs_", contrast, ".csv"))
}

**Extracting DMPs within Each DMR**

In [ ]:
# Extract DMPs within Each DMR
anno <- getAnnotation(IlluminaHumanMethylationEPICanno.ilm10b4.hg19)
for (contrast in contrasts) {
  dmrs <- dmr_results[[contrast]]$results
  dmr_granges <- extractRanges(dmrs, genome = "hg19")
  dmps <- dmp_list[[contrast]]
  # Add genomic coordinates to DMPs
  dmps_anno <- merge(dmps, anno, by.x = "row.names", by.y = "Name")
  dmps_granges <- GRanges(
    seqnames = dmps_anno$chr,
    ranges = IRanges(start = dmps_anno$pos, end = dmps_anno$pos),
    probe = dmps_anno$Row.names
  )
  
  # Find overlaps between DMRs and DMPs
  overlaps <- findOverlaps(dmr_granges, dmps_granges)
  
  # Save DMPs in each DMR to a folder
  dir.create(paste0("DMR_DMPs_", contrast), showWarnings = FALSE)
  for (i in 1:length(dmr_granges)) {
    dmr_name <- paste0("DMR_", i, "_", contrast)
    overlapping_dmps <- dmps_granges[subjectHits(overlaps[queryHits(overlaps) == i])]
    dmps_in_dmr <- dmps[dmps$Row.names %in% overlapping_dmps$probe, ]
    write.csv(
      dmps_in_dmr,
      file = paste0("DMR_DMPs_", contrast, "/", dmr_name, ".csv")
    )
  }
}

In [10]:
# results uploaded
#severe vs healthy samples DMRcate results
SN_325<-data.frame(readRDS("path"))
#severe vs mild samples DMRcate results
SM_325<-data.frame(readRDS("path"))
head(SN_325)

,seqnames,start,end,width,strand,no.cpgs,min_smoothed_fdr,Stouffer,HMFDR,Fisher,maxdiff,meandiff,overlapping.genes
,<fct>,<int>,<int>,<int>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,chr6,33215649,33217620,1972,*,42,1.157070e-61,2.416668e-52,6.529396e-10,3.321647e-79,-0.09887968,-0.0044655459,HCG25
2,chr6,31695415,31698899,3485,*,68,2.694147e-58,5.883650e-82,5.433656e-10,8.094708e-110,-0.11498157,-0.0027726130,"DDAH2, CLIC1"
3,chr6,33385056,33387205,2150,*,56,1.351952e-57,3.763078e-47,5.831644e-10,1.540231e-69,0.07737654,0.0033838782,CUTA
4,chr17,3539439,3540594,1156,*,19,1.430323e-55,1.305430e-29,2.304661e-11,1.893678e-41,-0.12629947,-0.0051858657,"SHPK, SHPK, CTNS"
5,chr6,29854631,29856938,2308,*,45,6.466200e-54,3.763759e-70,8.291570e-10,1.778023e-86,-0.11848446,0.0004962253,"HCG4P7, HLA-H"
6,chr6,32804435,32807374,2940,*,42,5.859055e-53,1.470068e-70,1.861365e-11,1.612606e-90,0.07981323,0.0048094082,"TAP2, TAP2"


In [ ]:
#SEVEREvsNEGATIVE
library(IlluminaHumanMethylationEPICanno.ilm10b4.hg19)
# Load annotation data
anno <- getAnnotation(IlluminaHumanMethylationEPICanno.ilm10b4.hg19)
m_values <- as.matrix(m_values)
annot <- cpg.annotate(
    object = m_values, 
    datatype = "array", 
    what = "M", 
    analysis.type = "differential", 
    design = design, 
    contrasts = TRUE, 
    cont.matrix = contrast_matrix, 
    coef = "SEVEREvsNEGATIVE", 
    arraytype = "EPICv1",
    anno = anno)

# Run DMRcate
dmrs_SN <- dmrcate(annot, lambda=500, C=2)
results_ranges_SN <- extractRanges(dmrs_SN)
results_ranges_SN
#saveRDS(results_ranges_SN, file = "results_ranges_SN.rds")

In [43]:
#SEVEREvsMILD
# Load annotation data
anno <- getAnnotation(IlluminaHumanMethylationEPICanno.ilm10b4.hg19)

m_values <- as.matrix(m_values)
annot <- cpg.annotate(
    object = m_values, 
    datatype = "array", 
    what = "M", 
    analysis.type = "differential", 
    design = design, 
    contrasts = TRUE, 
    cont.matrix = contrast_matrix, 
    coef = "SEVEREvsMILD",  # or "SEVEREvsNEGATIVE"
    arraytype = "EPICv1",
    anno = anno)

# Run DMRcate
dmrs <- dmrcate(annot, lambda=500, C=2)
# Extract and view results
results_ranges <- extractRanges(dmrs)
results_ranges
#saveRDS(results_ranges, file = "results_ranges.rds")

Your contrast returned 247961 individually significant probes. We recommend the default setting of pcutoff in dmrcate().

Warning message in S4Vectors:::normarg_mcols(mcols, Class, ans_len):
"You supplied metadata columns of length 768067 to set on an object of
  length 865859. However please note that the latter is not a multiple of
  the former."
Fitting chr1...

Fitting chr2...

Fitting chr3...

Fitting chr4...

Fitting chr5...

Fitting chr6...

Fitting chr7...

Fitting chr8...

Fitting chr9...

Fitting chr10...

Fitting chr11...

Fitting chr12...

Fitting chr13...

Fitting chr14...

Fitting chr15...

Fitting chr16...

Fitting chr17...

Fitting chr18...

Fitting chr19...

Fitting chr20...

Fitting chr21...

Fitting chr22...

Fitting chrX...

Fitting chrY...

Demarcating regions...

Done!

DMRcatedata not installed.
  Full functionality, documentation, and loading of data might not be possible without installing

loading from cache



GRanges object with 44061 ranges and 8 metadata columns:
          seqnames              ranges strand |   no.cpgs min_smoothed_fdr
             <Rle>           <IRanges>  <Rle> | <integer>        <numeric>
      [1]     chr6   29854740-29856938      * |        44      1.39106e-73
      [2]     chr6   31695970-31698698      * |        62      1.76047e-71
      [3]     chr6   33215649-33217620      * |        42      1.28692e-62
      [4]     chr6   32805142-32807374      * |        38      1.23415e-61
      [5]     chr6   32935354-32942808      * |       126      1.38635e-60
      ...      ...                 ...    ... .       ...              ...
  [44057]     chrX   16860753-16860764      * |         2      3.21186e-05
  [44058]     chr7     6120483-6120572      * |         4      3.22971e-05
  [44059]     chr3 100401270-100401284      * |         2      3.25085e-05
  [44060]     chr9 129885080-129885084      * |         2      3.25276e-05
  [44061]     chr7   54406974-54407004     